# End-to-End model building process for MNIST

This is a complete end-to-end build at MNIST with Theano + Keras + Python 2.7 on Ubuntu 16.04 + GTX 1080 Ti.

Using Data Augmentation and Batch Normalization.

Note: MNIST is great way to revise basics about CNNs because it's very fast to train (28x28 images) and there are plenty  benchmarks available on best approaches.
Also Keras contains a copy of MNIST.

In [1]:
import os, sys
#Create references to important directories we will use over and over
#current_dir = os.getcwd()

#Allow relative imports to directories above lesson3/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
#Import modules
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

#In Jupyter notebooks, we need to run this command before doing any plotting
%matplotlib inline

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


# Setup


In [3]:
batch_size = 64

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

#### Preprocessing: add missing color and OneHot

MNIST are greyscale images while  Keras expect RGB (so 3 color channels) images so we need to add an empty dimension (the missing "color channel") to MNIST to avoid errors.

In [5]:
X_test = np.expand_dims(X_test, 1)
X_train = np.expand_dims(X_train, 1)
X_train.shape

(60000, 1, 28, 28)

Also we need to *onehot* encode the labels (*y_values*) because they are actual real figures (0,1,2,3 etc), so *Softmax* can approximate the result with a very high value close to 1.

In [6]:
#examples
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [7]:
#Onehot encoding
y_train = onehot(y_train)
y_test = onehot(y_test)

y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

#### Normalization
We normalize the input by substracting the mean and dividing by the Standard Deviation


In [8]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [9]:
def norm_input(x): return (x-mean_px) / std_px

# 1. Linear Model

We need a linear model which needs to:
- normalize the input, 
- flatten it as a simple vector instead of an image,
- create a Dense layer with 10 outputs under *Softmax*.


In [10]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape = (1, 28, 28)),
        Flatten(),
        Dense(10, activation='softmax')    
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
lm = get_lin_model()

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [12]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 1, 28, 28)     0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 784)           0           lambda_1[0][0]                   
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            7850        flatten_1[0][0]                  
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
____________________________________________________________________________________________________


In [13]:
#Basic Gen without data augmentation
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size = batch_size)
test_batches = gen.flow(X_test, y_test, batch_size = batch_size)

In [14]:
# Let's do a first single epoch run with standard params, including lr=0.001
lm.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 8s - loss: 0.4287 - acc: 0.8747 - val_loss: 0.2963 - val_acc: 0.9158


In [15]:
lm.optimizer.lr=0.1

In [16]:
lm.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 8s - loss: 0.2983 - acc: 0.9139 - val_loss: 0.2810 - val_acc: 0.9193


In [17]:
lm.optimizer.lr=0.01

In [18]:
lm.fit_generator(batches, batches.n, nb_epoch=5,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
60000/60000 [==============================] - 9s - loss: 0.2850 - acc: 0.9195 - val_loss: 0.2798 - val_acc: 0.9202
Epoch 2/5
60000/60000 [==============================] - 10s - loss: 0.2782 - acc: 0.9215 - val_loss: 0.2819 - val_acc: 0.9210
Epoch 3/5
60000/60000 [==============================] - 9s - loss: 0.2723 - acc: 0.9235 - val_loss: 0.2852 - val_acc: 0.9210
Epoch 4/5
60000/60000 [==============================] - 8s - loss: 0.2689 - acc: 0.9248 - val_loss: 0.2789 - val_acc: 0.9202
Epoch 5/5
60000/60000 [==============================] - 8s - loss: 0.2664 - acc: 0.9260 - val_loss: 0.2788 - val_acc: 0.9218


# 2. Single dense layer

In [19]:
# Let's add one hidden layer fully-connected, like what people called "Neural Networks" in the 80-90's.
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape = (1, 28, 28)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')    
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
fc = get_fc_model()

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [21]:
#And same routine as before
fc.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 10s - loss: 0.1951 - acc: 0.9400 - val_loss: 0.1106 - val_acc: 0.9647


In [22]:
lm.optimizer.lr=0.1

In [23]:
fc.fit_generator(batches, batches.n, nb_epoch=6,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
60000/60000 [==============================] - 10s - loss: 0.0828 - acc: 0.9744 - val_loss: 0.0806 - val_acc: 0.9739
Epoch 2/5
60000/60000 [==============================] - 10s - loss: 0.0574 - acc: 0.9817 - val_loss: 0.0802 - val_acc: 0.9762
Epoch 3/5
60000/60000 [==============================] - 9s - loss: 0.0431 - acc: 0.9863 - val_loss: 0.0790 - val_acc: 0.9767
Epoch 4/5
60000/60000 [==============================] - 9s - loss: 0.0342 - acc: 0.9886 - val_loss: 0.0800 - val_acc: 0.9767
Epoch 5/5
60000/60000 [==============================] - 9s - loss: 0.0300 - acc: 0.9901 - val_loss: 0.0634 - val_acc: 0.9819


In [24]:
lm.optimizer.lr=0.01

In [25]:
fc.fit_generator(batches, batches.n, nb_epoch=8,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
60000/60000 [==============================] - 9s - loss: 0.0232 - acc: 0.9928 - val_loss: 0.0836 - val_acc: 0.9782
Epoch 2/5
60000/60000 [==============================] - 9s - loss: 0.0222 - acc: 0.9927 - val_loss: 0.0824 - val_acc: 0.9809
Epoch 3/5
60000/60000 [==============================] - 9s - loss: 0.0187 - acc: 0.9937 - val_loss: 0.0764 - val_acc: 0.9813
Epoch 4/5
60000/60000 [==============================] - 9s - loss: 0.0185 - acc: 0.9940 - val_loss: 0.0840 - val_acc: 0.9807
Epoch 5/5
60000/60000 [==============================] - 9s - loss: 0.0161 - acc: 0.9950 - val_loss: 0.0857 - val_acc: 0.9813


# 3. Basic 'VGG-style' CNN

In [26]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')        
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [27]:
model = get_model()

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [28]:
#And same routine as before
model.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 16s - loss: 0.1152 - acc: 0.9642 - val_loss: 0.0321 - val_acc: 0.9895


In [29]:
lm.optimizer.lr=0.1
model.fit_generator(batches, batches.n, nb_epoch=6,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
60000/60000 [==============================] - 15s - loss: 0.0346 - acc: 0.9894 - val_loss: 0.0242 - val_acc: 0.9925
Epoch 2/5
60000/60000 [==============================] - 15s - loss: 0.0246 - acc: 0.9923 - val_loss: 0.0269 - val_acc: 0.9925
Epoch 3/5
60000/60000 [==============================] - 15s - loss: 0.0185 - acc: 0.9946 - val_loss: 0.0229 - val_acc: 0.9930
Epoch 4/5
60000/60000 [==============================] - 16s - loss: 0.0173 - acc: 0.9945 - val_loss: 0.0291 - val_acc: 0.9921
Epoch 5/5
60000/60000 [==============================] - 15s - loss: 0.0126 - acc: 0.9959 - val_loss: 0.0341 - val_acc: 0.9908


In [30]:
lm.optimizer.lr=0.01
model.fit_generator(batches, batches.n, nb_epoch=8,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 15s - loss: 0.0114 - acc: 0.9967 - val_loss: 0.0267 - val_acc: 0.9930
Epoch 2/8
60000/60000 [==============================] - 15s - loss: 0.0099 - acc: 0.9970 - val_loss: 0.0260 - val_acc: 0.9947
Epoch 3/8
60000/60000 [==============================] - 15s - loss: 0.0084 - acc: 0.9973 - val_loss: 0.0469 - val_acc: 0.9892
Epoch 4/8
60000/60000 [==============================] - 15s - loss: 0.0087 - acc: 0.9972 - val_loss: 0.0260 - val_acc: 0.9929
Epoch 5/8
60000/60000 [==============================] - 15s - loss: 0.0068 - acc: 0.9979 - val_loss: 0.0234 - val_acc: 0.9938
Epoch 6/8
60000/60000 [==============================] - 15s - loss: 0.0072 - acc: 0.9977 - val_loss: 0.0318 - val_acc: 0.9921
Epoch 7/8
60000/60000 [==============================] - 15s - loss: 0.0048 - acc: 0.9985 - val_loss: 0.0301 - val_acc: 0.9945
Epoch 8/8
60000/60000 [==============================] - 16s - loss: 0.0071 - acc: 0.9976 - val_loss: 0.0354 - 

With lr=0.01, we see a drop in val_acc 0.9922 over train_acc:0.9979 so we are now clearly overfitting compared to lr=0.1

#### Which is great news: we now know we have a model which is complex enough to handle our data.
#### "*Start by overfitting* " said Jeremy Howard, then move to 5-steps dance...

##### Recall on 5-steps to reduce overfitting:
    1. Add more data (not always possible, e.g. Kaggle competitions)
    2. Use Data Augmentation (duplicate+tweak the images)
    3. Use architectures that generalize well
    4. Add regularization via Batch Normalization
    5. Reduce architecture complexity




# Data Augmentation

In [31]:
model = get_model()

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [45]:
# https://keras.io/preprocessing/image/#imagedatagenerator
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, height_shift_range=0.08,
                               zoom_range=0.08, shear_range=0.3) #, channel_shift_range=0.08)
batches = gen.flow(X_train, y_train, batch_size = batch_size)
test_batches = gen.flow(X_test, y_test, batch_size = batch_size)

In [33]:
#And same routine as before
model.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 17s - loss: 0.1816 - acc: 0.9424 - val_loss: 0.0605 - val_acc: 0.9824


In [34]:
lm.optimizer.lr=0.1
model.fit_generator(batches, batches.n, nb_epoch=6,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 17s - loss: 0.0626 - acc: 0.9804 - val_loss: 0.0492 - val_acc: 0.9838
Epoch 2/8
60000/60000 [==============================] - 17s - loss: 0.0481 - acc: 0.9852 - val_loss: 0.0449 - val_acc: 0.9854
Epoch 3/8
60000/60000 [==============================] - 17s - loss: 0.0420 - acc: 0.9865 - val_loss: 0.0306 - val_acc: 0.9897
Epoch 4/8
60000/60000 [==============================] - 17s - loss: 0.0368 - acc: 0.9887 - val_loss: 0.0413 - val_acc: 0.9863
Epoch 5/8
60000/60000 [==============================] - 16s - loss: 0.0357 - acc: 0.9894 - val_loss: 0.0362 - val_acc: 0.9880
Epoch 6/8
60000/60000 [==============================] - 16s - loss: 0.0317 - acc: 0.9904 - val_loss: 0.0386 - val_acc: 0.9881
Epoch 7/8
60000/60000 [==============================] - 17s - loss: 0.0302 - acc: 0.9905 - val_loss: 0.0334 - val_acc: 0.9901
Epoch 8/8
60000/60000 [==============================] - 17s - loss: 0.0287 - acc: 0.9908 - val_loss: 0.0394 - 

In [35]:
lm.optimizer.lr=0.01
model.fit_generator(batches, batches.n, nb_epoch=8,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 16s - loss: 0.0253 - acc: 0.9919 - val_loss: 0.0278 - val_acc: 0.9906
Epoch 2/8
60000/60000 [==============================] - 16s - loss: 0.0248 - acc: 0.9920 - val_loss: 0.0278 - val_acc: 0.9914
Epoch 3/8
60000/60000 [==============================] - 17s - loss: 0.0231 - acc: 0.9924 - val_loss: 0.0349 - val_acc: 0.9899
Epoch 4/8
60000/60000 [==============================] - 16s - loss: 0.0229 - acc: 0.9925 - val_loss: 0.0356 - val_acc: 0.9894
Epoch 5/8
60000/60000 [==============================] - 17s - loss: 0.0214 - acc: 0.9932 - val_loss: 0.0340 - val_acc: 0.9903
Epoch 6/8
60000/60000 [==============================] - 16s - loss: 0.0225 - acc: 0.9928 - val_loss: 0.0303 - val_acc: 0.9899
Epoch 7/8
60000/60000 [==============================] - 16s - loss: 0.0201 - acc: 0.9942 - val_loss: 0.0239 - val_acc: 0.9915
Epoch 8/8
60000/60000 [==============================] - 17s - loss: 0.0213 - acc: 0.9941 - val_loss: 0.0275 - 

In [36]:
lm.optimizer.lr=0.001
model.fit_generator(batches, batches.n, nb_epoch=10,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 16s - loss: 0.0183 - acc: 0.9941 - val_loss: 0.0291 - val_acc: 0.9919
Epoch 2/8
60000/60000 [==============================] - 16s - loss: 0.0203 - acc: 0.9934 - val_loss: 0.0214 - val_acc: 0.9938
Epoch 3/8
60000/60000 [==============================] - 16s - loss: 0.0185 - acc: 0.9942 - val_loss: 0.0354 - val_acc: 0.9903
Epoch 4/8
60000/60000 [==============================] - 18s - loss: 0.0183 - acc: 0.9944 - val_loss: 0.0277 - val_acc: 0.9922
Epoch 5/8
60000/60000 [==============================] - 17s - loss: 0.0175 - acc: 0.9943 - val_loss: 0.0312 - val_acc: 0.9923
Epoch 6/8
60000/60000 [==============================] - 17s - loss: 0.0182 - acc: 0.9942 - val_loss: 0.0223 - val_acc: 0.9941
Epoch 7/8
60000/60000 [==============================] - 17s - loss: 0.0176 - acc: 0.9944 - val_loss: 0.0278 - val_acc: 0.9913
Epoch 8/8
60000/60000 [==============================] - 17s - loss: 0.0161 - acc: 0.9946 - val_loss: 0.0234 - 

# BatchNormalization + Data Augmentation

In [37]:
def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(axis=1),
        Dense(512, activation='relu'),
        BatchNormalization(axis=1),
        Dense(10, activation='softmax')        
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [38]:
model = get_model_bn()

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [39]:
#And same routine as before
model.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 28s - loss: 0.1439 - acc: 0.9555 - val_loss: 0.0624 - val_acc: 0.9806


In [40]:
lm.optimizer.lr=0.1
model.fit_generator(batches, batches.n, nb_epoch=6,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 26s - loss: 0.0615 - acc: 0.9811 - val_loss: 0.0465 - val_acc: 0.9841
Epoch 2/8
60000/60000 [==============================] - 27s - loss: 0.0531 - acc: 0.9832 - val_loss: 0.0559 - val_acc: 0.9845
Epoch 3/8
60000/60000 [==============================] - 28s - loss: 0.0467 - acc: 0.9851 - val_loss: 0.0422 - val_acc: 0.9862
Epoch 4/8
60000/60000 [==============================] - 27s - loss: 0.0429 - acc: 0.9864 - val_loss: 0.0333 - val_acc: 0.9898
Epoch 5/8
60000/60000 [==============================] - 30s - loss: 0.0373 - acc: 0.9879 - val_loss: 0.0413 - val_acc: 0.9877
Epoch 6/8
60000/60000 [==============================] - 26s - loss: 0.0356 - acc: 0.9886 - val_loss: 0.0334 - val_acc: 0.9892
Epoch 7/8
60000/60000 [==============================] - 30s - loss: 0.0340 - acc: 0.9892 - val_loss: 0.0341 - val_acc: 0.9884
Epoch 8/8
60000/60000 [==============================] - 28s - loss: 0.0311 - acc: 0.9898 - val_loss: 0.0242 - 

In [41]:
lm.optimizer.lr=0.01
model.fit_generator(batches, batches.n, nb_epoch=8,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 26s - loss: 0.0315 - acc: 0.9897 - val_loss: 0.0278 - val_acc: 0.9899
Epoch 2/8
60000/60000 [==============================] - 26s - loss: 0.0297 - acc: 0.9909 - val_loss: 0.0300 - val_acc: 0.9903
Epoch 3/8
60000/60000 [==============================] - 26s - loss: 0.0276 - acc: 0.9914 - val_loss: 0.0241 - val_acc: 0.9914
Epoch 4/8
60000/60000 [==============================] - 26s - loss: 0.0255 - acc: 0.9923 - val_loss: 0.0247 - val_acc: 0.9916
Epoch 5/8
60000/60000 [==============================] - 26s - loss: 0.0241 - acc: 0.9922 - val_loss: 0.0272 - val_acc: 0.9905
Epoch 6/8
60000/60000 [==============================] - 26s - loss: 0.0245 - acc: 0.9922 - val_loss: 0.0222 - val_acc: 0.9935
Epoch 7/8
60000/60000 [==============================] - 26s - loss: 0.0226 - acc: 0.9931 - val_loss: 0.0220 - val_acc: 0.9935
Epoch 8/8
60000/60000 [==============================] - 26s - loss: 0.0231 - acc: 0.9927 - val_loss: 0.0323 - 

In [46]:
lm.optimizer.lr=0.001
model.fit_generator(batches, batches.n, nb_epoch=10,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 28s - loss: 0.0175 - acc: 0.9942 - val_loss: 0.0225 - val_acc: 0.9926
Epoch 2/8
60000/60000 [==============================] - 27s - loss: 0.0157 - acc: 0.9947 - val_loss: 0.0258 - val_acc: 0.9913
Epoch 3/8
60000/60000 [==============================] - 28s - loss: 0.0184 - acc: 0.9940 - val_loss: 0.0202 - val_acc: 0.9933
Epoch 4/8
60000/60000 [==============================] - 28s - loss: 0.0179 - acc: 0.9939 - val_loss: 0.0226 - val_acc: 0.9927
Epoch 5/8
60000/60000 [==============================] - 27s - loss: 0.0166 - acc: 0.9950 - val_loss: 0.0270 - val_acc: 0.9927
Epoch 6/8
60000/60000 [==============================] - 27s - loss: 0.0169 - acc: 0.9945 - val_loss: 0.0267 - val_acc: 0.9924
Epoch 7/8
60000/60000 [==============================] - 27s - loss: 0.0152 - acc: 0.9952 - val_loss: 0.0177 - val_acc: 0.9940
Epoch 8/8
60000/60000 [==============================] - 27s - loss: 0.0149 - acc: 0.9953 - val_loss: 0.0231 - 

# BatchNormalization + Dropout + Data Augmentation

In [49]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(axis=1),
        Dense(512, activation='relu'),
        BatchNormalization(axis=1),
        Dropout(0.5),
        Dense(10, activation='softmax')        
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [50]:
model = get_model_bn_do()

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [51]:
#And same routine as before
model.fit_generator(batches, batches.n, nb_epoch=1,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 27s - loss: 0.2250 - acc: 0.9322 - val_loss: 0.0729 - val_acc: 0.9785


In [52]:
lm.optimizer.lr=0.1
model.fit_generator(batches, batches.n, nb_epoch=6,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/6
60000/60000 [==============================] - 27s - loss: 0.0917 - acc: 0.9727 - val_loss: 0.0503 - val_acc: 0.9827
Epoch 2/6
60000/60000 [==============================] - 27s - loss: 0.0747 - acc: 0.9772 - val_loss: 0.0484 - val_acc: 0.9838
Epoch 3/6
60000/60000 [==============================] - 27s - loss: 0.0672 - acc: 0.9790 - val_loss: 0.0431 - val_acc: 0.9863
Epoch 4/6
60000/60000 [==============================] - 27s - loss: 0.0615 - acc: 0.9812 - val_loss: 0.0355 - val_acc: 0.9886
Epoch 5/6
60000/60000 [==============================] - 28s - loss: 0.0568 - acc: 0.9824 - val_loss: 0.0444 - val_acc: 0.9853
Epoch 6/6
60000/60000 [==============================] - 27s - loss: 0.0524 - acc: 0.9844 - val_loss: 0.0330 - val_acc: 0.9907


In [53]:
lm.optimizer.lr=0.01
model.fit_generator(batches, batches.n, nb_epoch=8,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 26s - loss: 0.0488 - acc: 0.9850 - val_loss: 0.0388 - val_acc: 0.9876
Epoch 2/8
60000/60000 [==============================] - 27s - loss: 0.0478 - acc: 0.9855 - val_loss: 0.0298 - val_acc: 0.9904
Epoch 3/8
60000/60000 [==============================] - 26s - loss: 0.0475 - acc: 0.9853 - val_loss: 0.0334 - val_acc: 0.9887
Epoch 4/8
60000/60000 [==============================] - 27s - loss: 0.0442 - acc: 0.9860 - val_loss: 0.0346 - val_acc: 0.9888
Epoch 5/8
60000/60000 [==============================] - 27s - loss: 0.0404 - acc: 0.9874 - val_loss: 0.0284 - val_acc: 0.9912
Epoch 6/8
60000/60000 [==============================] - 27s - loss: 0.0398 - acc: 0.9880 - val_loss: 0.0251 - val_acc: 0.9915
Epoch 7/8
60000/60000 [==============================] - 28s - loss: 0.0386 - acc: 0.9883 - val_loss: 0.0273 - val_acc: 0.9910
Epoch 8/8
60000/60000 [==============================] - 27s - loss: 0.0377 - acc: 0.9884 - val_loss: 0.0255 - 

In [54]:
lm.optimizer.lr=0.001
model.fit_generator(batches, batches.n, nb_epoch=10,
                validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/10
60000/60000 [==============================] - 26s - loss: 0.0347 - acc: 0.9891 - val_loss: 0.0310 - val_acc: 0.9903
Epoch 2/10
60000/60000 [==============================] - 26s - loss: 0.0361 - acc: 0.9892 - val_loss: 0.0238 - val_acc: 0.9931
Epoch 3/10
60000/60000 [==============================] - 28s - loss: 0.0359 - acc: 0.9894 - val_loss: 0.0221 - val_acc: 0.9931
Epoch 4/10
60000/60000 [==============================] - 26s - loss: 0.0317 - acc: 0.9905 - val_loss: 0.0262 - val_acc: 0.9920
Epoch 5/10
60000/60000 [==============================] - 27s - loss: 0.0322 - acc: 0.9903 - val_loss: 0.0268 - val_acc: 0.9926
Epoch 6/10
60000/60000 [==============================] - 27s - loss: 0.0301 - acc: 0.9905 - val_loss: 0.0229 - val_acc: 0.9926
Epoch 7/10
60000/60000 [==============================] - 28s - loss: 0.0306 - acc: 0.9909 - val_loss: 0.0262 - val_acc: 0.9917
Epoch 8/10
60000/60000 [==============================] - 26s - loss: 0.0305 - acc: 0.9910 - val_loss: 0

# Ensembling

In [55]:
def fit_model():
    model = get_model_bn_do()
    
    model.fit_generator(batches, batches.n, nb_epoch=1, verbose=0,
                validation_data=test_batches, nb_val_samples=test_batches.n)
    
    model.optimizer.lr = 0.1
    model.fit_generator(batches, batches.n, nb_epoch=6, verbose=0,
                validation_data=test_batches, nb_val_samples=test_batches.n)
    
    model.optimizer.lr = 0.01
    model.fit_generator(batches, batches.n, nb_epoch=8, verbose=0,
                validation_data=test_batches, nb_val_samples=test_batches.n)
    
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.n, nb_epoch=10, verbose=0,
                validation_data=test_batches, nb_val_samples=test_batches.n)
    
    return model
   
    

In [56]:
models = [fit_model() for i in range(6)]

/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_8 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_9 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/eric/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_10 and cannot be automatically inferred with the Theano bac

In [59]:
for i,m in enumerate(models):
    m.save_weights('cbb-mnist-' + str(i) + '.pkl')

In [66]:
evals = np.array([m.evaluate(X_test, y_test, batch_size = 256) for m in models])

 9472/10000 [===========================>..] - ETA: 0s

In [67]:
evals.mean(axis=0)

array([ 0.0135,  0.9956])

In [68]:
all_preds = np.stack([m.predict(X_test, batch_size = 256) for m in models])

In [69]:
all_preds.shape

(6, 10000, 10)

In [70]:
avg_preds = all_preds.mean(axis=0)

In [71]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()

array(0.9969000220298767, dtype=float32)